# Dendrite Toolkit


This toolkit is used to interact with a browser using Dendrite. [See Dendrite's docs here](https://docs.dendrite.systems)

With this toolkit, your AI applications and AI agent are able to use the internet, and all it's web services, just like a human by using websites!

The tools bundled within the `Dendrite` toolkit include:

- `Ask` (ask_page_with_browser) - Looks at the current page and answers a question about it
- `Extract` (extract_from_page_with_browser) - Extracts data from the current page based on a prompt and optional type specification
- `Fill` (fill_element_with_browser) - Fills an element on the page based on the provided prompt and value
- `FillFields` (fill_multiple_fields_with_browser) - Fills multiple fields on the page based on a dictionary of prompts and values
- `Goto` (goto_with_browser) - Navigate with the Dendrite browser to a given URL
- `Click` (click_element_with_browser) - Clicks an element on the page based on the provided prompt
- `Press` (press_key_with_browser) - Presses a keyboard key on the active page, optionally with modifier keys
- `Upload` (upload_files_with_browser) - Uploads one or more files to the page using a file chooser
- `Download` (download_file_with_browser) - Downloads a file from the page to the specified path

## Setup and Installation

You'll need to set up a free Dendrite account to get started: [Get your API key here](https://dendrite.systems/app)

After this, install Dendrite:

In [1]:
%pip install dendrite-sdk langchain-community

# If this is your first time installing Dendrite, you'll need to download the browser binaries too:
% dendrite install


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Simple Example

Let's see how we initialize the Dendrite Client and use to get some information from a website:

> Make sure you added your [free Dendrite API key](https://dendrite.systems/app) into .env file as DENDRITE_API_KEY

In [ ]:
from langchain_community.tools.dendrite import create_async_dendrite_client, Goto, Ask
from langchain_community.agent_toolkits import DendriteToolkit

# We start by creating the Dendrite Client, which will initialize a chromium browser.
# It's in this step we specify the browser options, such as which remote browser hosting provider to use or on what websites to be authenticated on.
client = create_async_dendrite_client()

# When we create the toolkit, we have to pass in the Dendrite client.
toolkit = DendriteToolkit.from_client(client)
tools = toolkit.get_tools()
print("All tools: ", tools)

# Let's try using the raw tools to load a webpage and extract some information from it.
goto_tool = Goto().from_client(client)
goto_tool.run({"url": "https://github.com/dendrite-systems/dendrite-python-sdk"})
ask_tool = Ask().from_client(client)
ask_tool.run({"prompt": "How many stars does this repo have?"})


[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/isaachershenson/Library/Caches/ms-playwright/chromium-1124/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=127.0.6533.17>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/isaachershenson/Library/Caches/ms-playwright/chromium-1124/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=127.0.6533.17>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/isaachershenson/Library/Caches/ms-playwright/chromium-1124/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=127.0.6533.17>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/isaachershenson/Library/Caches/ms-playwright/chromium-1124/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=127.0.6533.17>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/U

## Use within an Agent

Let's give the toolkit to a ReAct AI agent and have it summerize and download a github repo.

In [10]:
from langchain_community.agent_toolkits.dendrite import DendriteToolkit
from langchain_community.tools.dendrite.utils import (
    create_sync_dendrite_client,
)
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

prompt = hub.pull("hwchase17/openai-tools-agent")

# Authenticate on github (See more in the dendrite docs on how do do this)
client = create_sync_dendrite_client(auth="github.com")
toolkit = DendriteToolkit.from_client(client=client)
tools = toolkit.get_tools()

llm = ChatOpenAI(model="gpt-4o", temperature=0)
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

url = "https://github.com/dendrite-systems/dendrite-python-sdk"
command = {
    "input": f"go to {url}, summarize the repository and how many stars it has, then download the repo as a zip file at /tmp/repo.zip"
}
response = agent_executor.invoke(command)


/Users/isaachershenson/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
